# **!!!!非常重要!!!!**
### **請先在自己的Google drive建立 Aidea_data 這個名字的資料夾**
### **將所有hdf5檔和ckpt檔移到 Aidea_data 這個資料夾**
Aidea_data裡面包刮所有的hdf5格式的train data, valid data, test data還有一些checkpoint檔案(ckpt檔)

所有hdf5資料和所有ckpt檔下載路徑, 把這些hdf5資料和ckpt檔下載下來移到自己google drive所建立的 Aidea_data 資料夾裡面

確認自己的Aidea_data路徑為 : '/content/drive/MyDrive/Aidea_data'

https://drive.google.com/drive/folders/1WIwse2gGgs5t0CkjYdztatfQFpvR0WMf?usp=sharing

# **Mount google drive**

In [7]:
from google.colab import drive
import os
drive.mount('/content/drive')
os.chdir('/content/drive/MyDrive/Aidea_data')
!ls

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
banana_train.hdf5
banana_valid.hdf5
bareland_train.hdf5
bareland_valid.hdf5
carrot_train.hdf5
carrot_valid.hdf5
corn_train.hdf5
corn_valid.hdf5
dragonfruit_train.hdf5
dragonfruit_valid.hdf5
garlic_train.hdf5
garlic_valid.hdf5
guava_train.hdf5
guava_valid.hdf5
peanut_train.hdf5
peanut_valid.hdf5
pineapple_train.hdf5
pineapple_valid.hdf5
pumpkin_train.hdf5
pumpkin_valid.hdf5
rice_train.hdf5
rice_valid.hdf5
sample_3_Eff_b4_lrsche_imbalance_imbalance_best.ckpt
sample_kk_Effb3_imbalance_best.ckpt
sample_once_kk_Effb6_imbalance_best.ckpt
soybean_train.hdf5
soybean_valid.hdf5
sugarcane_train.hdf5
sugarcane_valid.hdf5
test_0.hdf5
test_1.hdf5
test_2.hdf5
test_3.hdf5
test_4.hdf5
test_5.hdf5
test_6.hdf5
test_7.hdf5
test_8.hdf5
test_9.hdf5
test_a.hdf5
test_b.hdf5
test_c.hdf5
test_d.hdf5
test_e.hdf5
test_f.hdf5
tomato_train.hdf5
tomato_valid.hdf5


# **Check GPU type**

In [2]:
!nvidia-smi

Fri May 20 03:43:58 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P0    23W / 300W |      0MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# **Import Package**

In [8]:
# Import necessary packages.
import numpy as np
import matplotlib.pyplot as plt
import os

import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torch.optim as optim
from torch.optim.lr_scheduler import LambdaLR
from torch.optim import Optimizer

from PIL import Image
# "ConcatDataset" and "Subset" are possibly useful when doing semi-supervised learning.
from torch.utils.data import ConcatDataset, DataLoader, Subset, Dataset
from torchvision.datasets import DatasetFolder, VisionDataset
from torch.utils.data.sampler import WeightedRandomSampler
import torchvision.models as models
import time
import copy
from torch.optim import lr_scheduler
from sklearn.metrics import confusion_matrix

# This is for the progress bar.
from tqdm.auto import tqdm
import random

# For dataset organization:
import os
import shutil
from shutil import copy2, move


import h5py
import numpy as np
import os
import matplotlib.pyplot as plt
import io
from PIL import Image

import h5py
import numpy as np
import os
import matplotlib.pyplot as plt
import io
from PIL import Image

device = "cuda" if torch.cuda.is_available() else "cpu"

# **Parameters settings**

In [9]:
num_class = 14
batch_size = 16
n_epochs = 15

# Pretrained Model settings for EfficientNet b3, b4, b6 :
# model_selection:  1 for efficientnet_b3 ; 2 for efficientnet_b4 ; 3 for efficientnet_b6
model_selection = 1

model1 = models.efficientnet_b3(pretrained=True)
model2 = models.efficientnet_b4(pretrained=True)
model3 = models.efficientnet_b6(pretrained=True)
model_list = [model1,model2,model3]

if model_selection==0 or model_selection>3:
  print('model list index out of range ! Please select index between 1 to 3')
  raise('error')
else:
  model = model_list[model_selection-1]

_exp_name = "sample_EffNetb3"   # Save the name of checkpoint
retrain = False  # Not retraining for default

if retrain:
  retrain_checkpoint = "/content/drive/MyDrive/Aidea_data/"+_exp_name+"_"+"best.ckpt"
else:
  retrain_checkpoint =[]

### 此處提供創建hdf5檔案的函式

In [ ]:
'''
save_path = '/content/drive/MyDrive/Aidea_data/carrot.hdf5'   # e.g : save path for created hdf5 file with name carrot.hdf5
base = "/content/drive/MyDrive/Aidea_data/train_data/carrot"  # e.g : Carrot Image data storage path

# Create hdf5 file function :
def create_dirset(base,dir_image):
  for k in dir_image:   # find all images inside a.
    img_path = os.path.join(base, k)
    #print('image size: %d bytes'%os.path.getsize(img_path))
    with open(img_path, 'rb') as img_f:  # open images as python binary
      binary_data = img_f.read()

    binary_data_np = np.asarray(binary_data)

    dset = hf.create_dataset(k, data=binary_data_np) # save it in the subgroup. each a-subgroup contains all the images.
  return dset

dir=os.listdir(base)
#dir.pop()
hf = h5py.File(save_path, 'a') # open a hdf5 file
dset = create_dirset(base,dir)
hf.close()  # close the hdf5 file
'''

# **Random Seed Settings**

In [10]:
from numpy.ma.core import true_divide
myseed = 7777  # set a random seed for reproducibility
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = True
np.random.seed(myseed)
torch.manual_seed(myseed)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(myseed)

# **Copy hdf5(image format) file from Google Drive**

In [6]:
!cp -r /content/drive/MyDrive/Aidea_data/banana_train.hdf5 /content
!cp -r /content/drive/MyDrive/Aidea_data/banana_valid.hdf5 /content

!cp -r /content/drive/MyDrive/Aidea_data/bareland_train.hdf5 /content
!cp -r /content/drive/MyDrive/Aidea_data/bareland_valid.hdf5 /content

!cp -r /content/drive/MyDrive/Aidea_data/carrot_train.hdf5 /content
!cp -r /content/drive/MyDrive/Aidea_data/carrot_valid.hdf5 /content

!cp -r /content/drive/MyDrive/Aidea_data/corn_train.hdf5 /content
!cp -r /content/drive/MyDrive/Aidea_data/corn_valid.hdf5 /content

!cp -r /content/drive/MyDrive/Aidea_data/dragonfruit_train.hdf5 /content
!cp -r /content/drive/MyDrive/Aidea_data/dragonfruit_valid.hdf5 /content

!cp -r /content/drive/MyDrive/Aidea_data/garlic_train.hdf5 /content
!cp -r /content/drive/MyDrive/Aidea_data/garlic_valid.hdf5 /content

!cp -r /content/drive/MyDrive/Aidea_data/guava_train.hdf5 /content
!cp -r /content/drive/MyDrive/Aidea_data/guava_valid.hdf5 /content

!cp -r /content/drive/MyDrive/Aidea_data/peanut_train.hdf5 /content
!cp -r /content/drive/MyDrive/Aidea_data/peanut_valid.hdf5 /content

!cp -r /content/drive/MyDrive/Aidea_data/pineapple_train.hdf5 /content
!cp -r /content/drive/MyDrive/Aidea_data/pineapple_valid.hdf5 /content

!cp -r /content/drive/MyDrive/Aidea_data/pumpkin_train.hdf5 /content
!cp -r /content/drive/MyDrive/Aidea_data/pumpkin_valid.hdf5 /content

!cp -r /content/drive/MyDrive/Aidea_data/rice_train.hdf5 /content
!cp -r /content/drive/MyDrive/Aidea_data/rice_valid.hdf5 /content

!cp -r /content/drive/MyDrive/Aidea_data/soybean_train.hdf5 /content
!cp -r /content/drive/MyDrive/Aidea_data/soybean_valid.hdf5 /content

!cp -r /content/drive/MyDrive/Aidea_data/sugarcane_train.hdf5 /content
!cp -r /content/drive/MyDrive/Aidea_data/sugarcane_valid.hdf5 /content

!cp -r /content/drive/MyDrive/Aidea_data/tomato_train.hdf5 /content
!cp -r /content/drive/MyDrive/Aidea_data/tomato_valid.hdf5 /content


# **Data augmentation**

In [11]:
test_tfm = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

train_tfm = transforms.Compose([
    # Resize the image into a fixed shape (height = width = 128)
    # You may add some transforms here.
    transforms.RandomRotation(40),
    transforms.RandomAffine(degrees=0, translate=(0.2, 0.2), shear=0.2),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.Resize((224, 224)),
    transforms.GaussianBlur(3,sigma=(0.1, 2.0)),
    # ToTensor() should be the last one of the transforms.
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# However, it is also possible to use augmentation in the testing phase.
# You may use train_tfm to produce a variety of images and then test using ensemble methods
'''
train_tfm = transforms.Compose([
    # Resize the image into a fixed shape (height = width = 128)
    # You may add some transforms here.
    #transforms.ColorJitter(brightness=0.2, contrast=0, saturation=0, hue=0.1),
    transforms.RandomRotation(40),
    #transforms.RandomPerspective(distortion_scale=0.5, p=0.5),
    transforms.RandomAffine(degrees=0, translate=(0.2, 0.2), shear=0.2),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomVerticalFlip(p=0.5),
    #transforms.Resize((224, 224)),
    transforms.RandomResizedCrop(size=(224, 224)),
    #transforms.RandomCrop(size=(128,128)),
    #transforms.RandomEqualize(p=0.5),
    #transforms.RandomAdjustSharpness(sharpness_factor=2, p=0.5),
    #transforms.GaussianBlur(3,sigma=(0.1, 2.0)),
    # ToTensor() should be the last one of the transforms.
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])
'''

'\ntrain_tfm = transforms.Compose([\n    # Resize the image into a fixed shape (height = width = 128)\n    # You may add some transforms here.\n    #transforms.ColorJitter(brightness=0.2, contrast=0, saturation=0, hue=0.1),\n    transforms.RandomRotation(40),\n    #transforms.RandomPerspective(distortion_scale=0.5, p=0.5),\n    transforms.RandomAffine(degrees=0, translate=(0.2, 0.2), shear=0.2),\n    transforms.RandomHorizontalFlip(p=0.5),\n    transforms.RandomVerticalFlip(p=0.5),\n    #transforms.Resize((224, 224)),\n    transforms.RandomResizedCrop(size=(224, 224)),\n    #transforms.RandomCrop(size=(128,128)),\n    #transforms.RandomEqualize(p=0.5),\n    #transforms.RandomAdjustSharpness(sharpness_factor=2, p=0.5),\n    #transforms.GaussianBlur(3,sigma=(0.1, 2.0)),\n    # ToTensor() should be the last one of the transforms.\n    transforms.ToTensor(),\n    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])\n])\n'

# **Dataset Preparing**
####label = 0 -> banana
####label = 1 -> bareland
####label = 2 -> carrot
####label = 3 -> corn
####label = 4 -> dragonfruit
####label = 5 -> garlic
####label = 6 -> guava
####label = 7 -> peanut
####label = 8 -> pineapple
####label = 9 -> pumpkin
####label = 10 -> rice
####label = 11 -> soybean
####label = 12 -> sugarcane
####label = 13 -> tomato

In [12]:
class AIDEA_Dataset(Dataset):

    def __init__(self,path,tfm=test_tfm,types=None,files = None, file_name=True):
        super(AIDEA_Dataset).__init__()
        self.path = path
        self.hf = h5py.File(path, 'r')
        self.files = list(h5py.File(path, 'r').keys())
        #self.files = sorted([os.path.join(path,x) for x in os.listdir(path) if x.endswith(".JPG")])
        if files != None:
            self.files = files
        self.transform = tfm
        self.types = types
        self.file_name = file_name
  
    def __len__(self):
        return len(self.files)
  
    def __getitem__(self,idx):
        fname = self.files[idx]
        data = np.array(self.hf[fname])
        im = Image.open(io.BytesIO(data))
        #im = Image.open(fname)
        im = self.transform(im)
        #im = self.data[idx]
        #try:
       # label = int(fname.split("/")[-1].split("_")[0])
        if self.types =="banana":
            label = 0
        elif self.types =="bareland":
            label = 1
        elif self.types =="carrot":
            label = 2
        elif self.types =="corn":
            label = 3
        elif self.types =="dragonfruit":
            label = 4
        elif self.types =="garlic":
            label = 5
        elif self.types =="guava":
            label = 6
        elif self.types =="peanut":
            label = 7
        elif self.types =="pineapple":
            label = 8
        elif self.types =="pumpkin":
            label = 9
        elif self.types =="rice":
            label = 10
        elif self.types =="soybean":
            label = 11
        elif self.types =="sugarcane":
            label = 12
        elif self.types =="tomato":
            label = 13
        else:
            label = -1

        if self.file_name:
          return im,label,fname
        else:
          return im,label

In [13]:
trainDir_banana = "/content/banana_train.hdf5"
trainDir_bareland = "/content/bareland_train.hdf5"
trainDir_carrot = "/content/carrot_train.hdf5"
trainDir_corn = "/content/corn_train.hdf5"
trainDir_dragonfruit = "/content/dragonfruit_train.hdf5"
trainDir_garlic = "/content/garlic_train.hdf5"
trainDir_guava = "/content/guava_train.hdf5"
trainDir_peanut = "/content/peanut_train.hdf5"
trainDir_pineapple = "/content/pineapple_train.hdf5"
trainDir_pumpkin = "/content/pumpkin_train.hdf5"
trainDir_rice = "/content/rice_train.hdf5"
trainDir_soybean = "/content/soybean_train.hdf5"
trainDir_sugarcane = "/content/sugarcane_train.hdf5"
trainDir_tomato = "/content/tomato_train.hdf5"


validDir_banana = "/content/banana_valid.hdf5"
validDir_bareland = "/content/bareland_valid.hdf5"
validDir_carrot = "/content/carrot_valid.hdf5"
validDir_corn = "/content/corn_valid.hdf5"
validDir_dragonfruit = "/content/dragonfruit_valid.hdf5"
validDir_garlic = "/content/garlic_valid.hdf5"
validDir_guava = "/content/guava_valid.hdf5"
validDir_peanut = "/content/peanut_valid.hdf5"
validDir_pineapple = "/content/pineapple_valid.hdf5"
validDir_pumpkin = "/content/pumpkin_valid.hdf5"
validDir_rice = "/content/rice_valid.hdf5"
validDir_soybean = "/content/soybean_valid.hdf5"
validDir_sugarcane = "/content/sugarcane_valid.hdf5"
validDir_tomato = "/content/tomato_valid.hdf5"


trainset_banana = AIDEA_Dataset(trainDir_banana, tfm=train_tfm, types="banana")
trainset_bareland = AIDEA_Dataset(trainDir_bareland, tfm=train_tfm, types="bareland")
trainset_carrot = AIDEA_Dataset(trainDir_carrot, tfm=train_tfm, types="carrot")
trainset_corn = AIDEA_Dataset(trainDir_corn, tfm=train_tfm, types="corn")
trainset_dragonfruit = AIDEA_Dataset(trainDir_dragonfruit, tfm=train_tfm, types="dragonfruit")
trainset_garlic = AIDEA_Dataset(trainDir_garlic, tfm=train_tfm, types="garlic")
trainset_guava = AIDEA_Dataset(trainDir_guava, tfm=train_tfm, types="guava")
trainset_peanut = AIDEA_Dataset(trainDir_peanut, tfm=train_tfm, types="peanut")
trainset_pineapple = AIDEA_Dataset(trainDir_pineapple, tfm=train_tfm, types="pineapple")
trainset_pumpkin = AIDEA_Dataset(trainDir_pumpkin, tfm=train_tfm, types="pumpkin")
trainset_rice = AIDEA_Dataset(trainDir_rice, tfm=train_tfm, types="rice")
trainset_soybean = AIDEA_Dataset(trainDir_soybean, tfm=train_tfm, types="soybean")
trainset_sugarcane = AIDEA_Dataset(trainDir_sugarcane, tfm=train_tfm, types="sugarcane")
trainset_tomato = AIDEA_Dataset(trainDir_tomato, tfm=train_tfm, types="tomato")

validset_banana = AIDEA_Dataset(validDir_banana, tfm=test_tfm, types="banana")
validset_bareland = AIDEA_Dataset(validDir_bareland, tfm=test_tfm, types="bareland")
validset_carrot = AIDEA_Dataset(validDir_carrot, tfm=test_tfm, types="carrot")
validset_corn = AIDEA_Dataset(validDir_corn, tfm=test_tfm, types="corn")
validset_dragonfruit = AIDEA_Dataset(validDir_dragonfruit, tfm=test_tfm, types="dragonfruit")
validset_garlic = AIDEA_Dataset(validDir_garlic, tfm=test_tfm, types="garlic")
validset_guava = AIDEA_Dataset(validDir_guava, tfm=test_tfm, types="guava")
validset_peanut = AIDEA_Dataset(validDir_peanut, tfm=test_tfm, types="peanut")
validset_pineapple = AIDEA_Dataset(validDir_pineapple, tfm=test_tfm, types="pineapple")
validset_pumpkin = AIDEA_Dataset(validDir_pumpkin, tfm=test_tfm, types="pumpkin")
validset_rice = AIDEA_Dataset(validDir_rice, tfm=test_tfm, types="rice")
validset_soybean = AIDEA_Dataset(validDir_soybean, tfm=test_tfm, types="soybean")
validset_sugarcane = AIDEA_Dataset(validDir_sugarcane, tfm=test_tfm, types="sugarcane")
validset_tomato = AIDEA_Dataset(validDir_tomato, tfm=test_tfm, types="tomato")

trainset_con = ConcatDataset([trainset_banana, trainset_bareland, trainset_carrot, trainset_corn,\
                trainset_dragonfruit, trainset_garlic, trainset_guava,\
                trainset_peanut, trainset_pineapple, trainset_pumpkin, trainset_rice,\
                trainset_soybean, trainset_sugarcane, trainset_tomato])
validset_con = ConcatDataset([validset_banana, validset_bareland, validset_carrot, validset_corn, \
                validset_dragonfruit, validset_garlic, validset_guava, \
                validset_peanut, validset_pineapple, validset_pumpkin, validset_rice, \
                validset_soybean, validset_sugarcane, validset_tomato])

### Cope with data imbalance with WeightedRandomSampler

In [14]:
train_label = [0]*len(trainset_banana)+[1]*len(trainset_bareland)+[2]*len(trainset_carrot)+[3]*len(trainset_corn)+[4]*len(trainset_dragonfruit)+\
        [5]*len(trainset_garlic)+[6]*len(trainset_guava)+[7]*len(trainset_peanut)+[8]*len(trainset_pineapple)+[9]*len(trainset_pumpkin)+\
        [10]*len(trainset_rice)+[11]*len(trainset_soybean)+[12]*len(trainset_sugarcane)+[13]*len(trainset_tomato)
valid_label = [0]*len(validset_banana)+[1]*len(validset_bareland)+[2]*len(validset_carrot)+[3]*len(validset_corn)+[4]*len(validset_dragonfruit)+\
        [5]*len(validset_garlic)+[6]*len(validset_guava)+[7]*len(validset_peanut)+[8]*len(validset_pineapple)+[9]*len(validset_pumpkin)+\
        [10]*len(validset_rice)+[11]*len(validset_soybean)+[12]*len(validset_sugarcane)+[13]*len(validset_tomato)
print(len(train_label))
print(len(valid_label))

def balance_prob(all_gold_lablel_ids):
    unique_label_ids = list(set(all_gold_lablel_ids))    
    label_probs = []
    for label_id in range(len(unique_label_ids)):        
        label_id_count = all_gold_lablel_ids.count(label_id)
        label_probs.append(1./label_id_count)
    dataset_element_weights = [] # each element prob
    for label_id in all_gold_lablel_ids:                
        dataset_element_weights.append(label_probs[label_id])
    return dataset_element_weights
balance_train_prob = balance_prob(train_label)
balance_valid_prob = balance_prob(valid_label)
#print(balance_train_prob)

48179
12054


In [15]:
sampler_train = WeightedRandomSampler(weights=balance_train_prob,num_samples=len(trainset_con),replacement=True)
sampler_valid = WeightedRandomSampler(weights=balance_valid_prob,num_samples=len(validset_con),replacement=True)
train_loader = DataLoader(trainset_con, sampler=sampler_train ,batch_size=batch_size, num_workers=4, pin_memory=True)
valid_loader = DataLoader(validset_con, shuffle=True, batch_size=batch_size, num_workers=4, pin_memory=True)

print("Length of train data is ", len(trainset_con))
print("Length of valid data is ", len(validset_con))

Length of train data is  48179
Length of valid data is  12054


# **Function**

In [16]:
def data_loader(train_loader,valid_loader,phase):
  if phase=='train':
    data_load = train_loader
  elif phase=='val':
    data_load = valid_loader
  else:
    data_load=None
  return data_load
def data_size(train_loader,valid_loader,phase):
  if phase=='train':
    data_size_ = len(train_loader)
  elif phase=='val':
    data_size_ = len(valid_loader)
  else:
    data_load=0
  return data_size_

# Confusion matrix
def confusion_mat(label,predict):
    cnf_matrix = confusion_matrix(label, predict)
    return cnf_matrix

# Calculate all metrics from confusion matrix
def calculate_metric(cm , num_classes = num_class ,dataloader=range(13)):
    TP = np.diag(cm)
    FP = np.sum(cm, axis=0) - TP
    FN = np.sum(cm, axis=1) - TP
    
    TN = []
    precision = []
    recall = []
    #f1_score = []
    for i in range(num_classes):
        temp = np.delete(cm, i, 0)    # delete ith row
        temp = np.delete(temp, i, 1)  # delete ith column
        TN.append(sum(sum(temp)))     # True negative per class
        
        precision.append(TP[i]/(TP[i]+FP[i]) if (TP[i]+FP[i])>0. else 0.)
        recall.append(TP[i]/(TP[i]+FN[i]) if (TP[i]+FN[i])>0. else 0.)
    p = np.array(precision)
    r = np.array(recall)
    #print((2*p*r)/(p+r))
    f1_score = []
    for i in range(num_classes):
        f1_score.append((2.*p[i]*r[i])/(p[i]+r[i]) if (p[i]+r[i])>0. else 0.)
    WP = np.sum(p*(TP+FN))/len(dataloader)
    
    TP = TP.tolist()    # True positive per class
    FP = FP.tolist()    # False positive per class
    FN = FN.tolist()    # False negative per class
    p = p.tolist() # precision per class
    r = r.tolist() # Recall per class  
    #TN = np.array(TN)
    
    return TP,FP,FN,TN,p,r,f1_score,WP
class focal_loss(nn.Module):
  def __init__(self, gamma=2):
    super(focal_loss, self).__init__()
    self.gamma = gamma
    self.ce = nn.CrossEntropyLoss()
  def forward(self, input, label):
    logp = self.ce(input, label)
    p = torch.exp(-logp)
    loss = (1-p)**self.gamma * logp
    return loss.mean()

# **Training**

In [21]:
patience = 300

# Transfer learning for training the last classifier:
num_ftrs = model.classifier[1].in_features
model.classifier[1] = nn.Linear(num_ftrs, num_class)

# Loss function : Using Cross Entropy Loss
if model_selection == 3:   # For efficient Net b6
  criterion = focal_loss()
else: # For efficient Net b3, b4
  criterion = nn.CrossEntropyLoss()

if retrain:
  checkpoint = torch.load(retrain_checkpoint)
  model.load_state_dict(checkpoint['model_state_dict'])
  model = model.to(device)
  optimizer = torch.optim.AdamW(model.parameters(), lr=0.001, weight_decay=1e-5)
  optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
  epochs = checkpoint['epoch']
else:
  model = model.to(device)
  optimizer = torch.optim.AdamW(model.parameters(), lr=0.001, weight_decay=1e-5)
  epochs = 0


# Decay LR by a factor of 0.1 every 5 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)
#exp_lr_scheduler = lr_scheduler.MultiStepLR(optimizer, milestones=[3,6,3], gamma=0.1)

stale = 0
best_acc = 0
WP_best = 0

for epoch in range(epochs, n_epochs):
        # ---------- Training ----------
        # Make sure the model is in train mode before training.
        model.train()

        # These are used to record information in training.
        train_loss = []
        train_accs = []
        record_logits = []
        record_label = []
        for batch in tqdm(train_loader):
            # A batch consists of image data and corresponding labels.
            imgs, labels, _ = batch
            record_label = record_label + labels.tolist()

            # Forward the data. (Make sure data and model are on the same device.)           
            logits = model(imgs.to(device))
            # Calculate the cross-entropy loss.
            # We don't need to apply softmax before computing cross-entropy as it is done automatically.
            loss = criterion(logits, labels.to(device))
            # Gradients stored in the parameters in the previous step should be cleared out first.
            optimizer.zero_grad()        
            # Compute the gradients for parameters.
            loss.backward()
            # Clip the gradient norms for stable training.
            grad_norm = nn.utils.clip_grad_norm_(model.parameters(), max_norm=10)
            # Update the parameters with computed gradients.
            optimizer.step()
            # Compute the accuracy for current batch.
            acc = (logits.argmax(dim=-1) == labels.to(device)).float().mean()
            record_logits = record_logits + logits.argmax(dim=-1).tolist()
            # Record the loss and accuracy.
            train_loss.append(loss.item())
            train_accs.append(acc)
        exp_lr_scheduler.step()
        # Confusion Matrix :
        cmx = confusion_mat(record_label,record_logits)
        # Calculate Metrics :
        _,_,_,_,_,_,f1_score,WP = calculate_metric(cmx,num_classes = num_class ,dataloader=trainset_con)
        train_loss = sum(train_loss) / len(train_loss)
        train_acc = sum(train_accs) / len(train_accs)   
        # Print the information.
        print(f"[ Train | {epoch + 1:03d}/{n_epochs:03d} ] loss = {train_loss:.5f}")
        print("f1_score per class [ banana, bareland, carrot, corn, dragonfruit, garlic, guava, peanut, pineapple, pumpkin, rice, soybean, sugarcane, tomato ] is\n ",\
            f1_score)
        print(f"WP = {WP:.5f}")

        # ---------- Validation ----------
        # Make sure the model is in eval mode so that some modules like dropout are disabled and work normally.
        model.eval()

        # These are used to record information in validation.
        valid_loss = []
        valid_accs = []
        record_logits = []
        record_label = []
        # Iterate the validation set by batches.
        for batch in tqdm(valid_loader):
            # A batch consists of image data and corresponding labels.
            imgs, labels, _ = batch
            record_label = record_label + labels.tolist()

            # We don't need gradient in validation.
            # Using torch.no_grad() accelerates the forward process.
            with torch.no_grad():
                logits = model(imgs.to(device))
            # We can still compute the loss (but not the gradient).
            loss = criterion(logits, labels.to(device))
            # Compute the accuracy for current batch.
            acc = (logits.argmax(dim=-1) == labels.to(device)).float().mean()
            record_logits = record_logits + logits.argmax(dim=-1).tolist()
            # Record the loss and accuracy.
            valid_loss.append(loss.item())
            valid_accs.append(acc)
            #break
        # The average loss and accuracy for entire validation set is the average of the recorded values.
        valid_loss = sum(valid_loss) / len(valid_loss)
        valid_acc = sum(valid_accs) / len(valid_accs)
        # Confusion Matrix :
        cmx = confusion_mat(record_label,record_logits)
        # Calculate Metrics :
        _,_,_,_,_,_,f1_score,WP = calculate_metric(cmx,num_classes = num_class ,dataloader=validset_con)

        # Print the information.
        #print(f"[ Valid | {epoch + 1:03d}/{n_epochs:03d} ] loss = {valid_loss:.5f}, acc = {valid_acc:.5f}")
        print(f"[ Valid | {epoch + 1:03d}/{n_epochs:03d} ] loss = {valid_loss:.5f}")
        print("f1_score per class [ banana, bareland, carrot, corn, dragonfruit, garlic, guava, peanut, pineapple, pumpkin, rice, soybean, sugarcane, tomato ] is\n ",\
            f1_score)
        print(f"WP = {WP:.5f}")

        # update logs
        if WP > WP_best:
            with open(f"./{_exp_name}_log.txt","a"):
                print(f"[ Valid | {epoch + 1:03d}/{n_epochs:03d} ] loss = {valid_loss:.5f}, acc = {valid_acc:.5f} -> best")
        else:
            with open(f"./{_exp_name}_log.txt","a"):
                print(f"[ Valid | {epoch + 1:03d}/{n_epochs:03d} ] loss = {valid_loss:.5f}, acc = {valid_acc:.5f}")


        # save models
        if WP > WP_best:
            print(f"Best model found at epoch {epoch}, saving model")
            #os.chdir('/content/drive/MyDrive/Aidea')
            torch.save({'epoch': epoch+1,
                  'model_state_dict': model.state_dict(),
                  'optimizer_state_dict': optimizer.state_dict(),
                  'loss':criterion,
                  }, f"{_exp_name}_best.ckpt") # only save best to prevent output memory exceed error
            #os.chdir('/content')
            WP_best = WP
            stale = 0
        else:
            stale += 1
            if stale > patience:
                print(f"No improvment {patience} consecutive epochs, early stopping")
                break

  0%|          | 0/3012 [00:00<?, ?it/s]

[ Train | 001/015 ] loss = 0.48608
f1_score per class [ banana, bareland, carrot, corn, dragonfruit, garlic, guava, peanut, pineapple, pumpkin, rice, soybean, sugarcane, tomato ] is
  [0.9247001011706893, 0.9134475747894278, 0.7799271096159237, 0.8145487202514594, 0.9134958382877527, 0.8464104423495286, 0.8876974351543254, 0.6762546816479401, 0.9039104521005961, 0.8421950526016491, 0.9087703834288233, 0.6372377622377622, 0.8687923250564334, 0.8486966652104266]
WP = 0.83963


  0%|          | 0/754 [00:00<?, ?it/s]

[ Valid | 001/015 ] loss = 0.61667
f1_score per class [ banana, bareland, carrot, corn, dragonfruit, garlic, guava, peanut, pineapple, pumpkin, rice, soybean, sugarcane, tomato ] is
  [0.967527060782681, 0.7906536154089043, 0.8161290322580644, 0.9295962629295962, 0.9445378151260505, 0.9242916860195076, 0.96045197740113, 0.7965976331360947, 0.9615713065755764, 0.851985559566787, 0.9579540528825314, 0.5698650127690624, 0.8453333333333333, 0.7807486631016042]
WP = 0.90901
[ Valid | 001/015 ] loss = 0.61667, acc = 0.85428 -> best
Best model found at epoch 0, saving model


  0%|          | 0/3012 [00:00<?, ?it/s]

...

[ Valid | 014/015 ] loss = 0.06523
f1_score per class [ banana, bareland, carrot, corn, dragonfruit, garlic, guava, peanut, pineapple, pumpkin, rice, soybean, sugarcane, tomato ] is
  [0.9967320261437909, 0.9881547775730456, 0.9682804674457429, 0.9854872149274361, 0.9880749574105621, 0.9877729257641921, 0.9915730337078652, 0.9698699651125912, 0.9945765540258658, 0.95906432748538, 0.9920335429769392, 0.9386323787258913, 0.9763313609467456, 0.9649122807017544]
WP = 0.98175
[ Valid | 014/015 ] loss = 0.06523, acc = 0.98168 -> best
Best model found at epoch 13, saving model


  0%|          | 0/3012 [00:00<?, ?it/s]

[ Train | 015/015 ] loss = 0.03168
f1_score per class [ banana, bareland, carrot, corn, dragonfruit, garlic, guava, peanut, pineapple, pumpkin, rice, soybean, sugarcane, tomato ] is
  [0.9963012279923065, 0.9919121894858463, 0.9907273254129237, 0.9867056245434623, 0.9987126305249607, 0.9885590152063722, 0.9968281430219146, 0.9606716747680072, 0.9966767808120214, 0.9964850615114236, 0.9944932281589521, 0.9569102176106068, 0.9957989280023178, 0.9964270401600684]
WP = 0.98910


  0%|          | 0/754 [00:00<?, ?it/s]

[ Valid | 015/015 ] loss = 0.06778
f1_score per class [ banana, bareland, carrot, corn, dragonfruit, garlic, guava, peanut, pineapple, pumpkin, rice, soybean, sugarcane, tomato ] is
  [0.9967320261437909, 0.9878883622959452, 0.9605263157894736, 0.9852081183350533, 0.9864406779661017, 0.9860262008733625, 0.9915492957746479, 0.9674329501915709, 0.9924749163879598, 0.955512572533849, 0.9912023460410557, 0.9345029239766082, 0.9806835066864784, 0.9652173913043478]
WP = 0.98047
[ Valid | 015/015 ] loss = 0.06778, acc = 0.98035
